In [ ]:
import torch
import torch.nn as nn

# Modelo de Política: mapeia o estado para uma distribuição de probabilidade sobre as ações
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim),
            nn.Softmax(dim=-1)  # Gera uma distribuição de probabilidade sobre as ações
        )
    
    def forward(self, x):
        return self.fc(x)

# Função para calcular os retornos (recompensas descontadas) com normalização
class RewardNetwork(nn.Module):
    def __init__(self, state_dim, group_size):
        super(RewardNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear((state_dim, group_size), 128),
            nn.ReLU(),
            nn.Tanh(128, 1) #ver qual é a funcao de recompensa do ambiente
        )
    
    def forward(self, x):
        return self.fc(x)


In [ ]:
import gymnasium as gym
import numpy as np

def collect_episodes(num_episodes=10):
    """
    Coleta num_episodes episódios do ambiente CartPole-v1.
    
    Cada episódio é uma lista de transições onde cada transição é uma tupla:
    (estado, ação, recompensa, próximo estado, done, truncated).
    
    Retorna:
        np.array: Array de episódios (cada episódio é um np.array de transições).
    """
    # Cria o ambiente
    env = gym.make('CartPole-v1')
    episodes_data = []

    # Coleta os episódios
    for ep in range(num_episodes):
        episode = []
        observation, info = env.reset()
        done = False
        truncated = False
        
        while not (done or truncated):
            # Seleciona uma ação aleatória
            action = env.action_space.sample()
            next_observation, reward, done, truncated, info = env.step(action)
            # Armazena a transição
            transition = (observation, action, reward, next_observation, done, truncated)
            episode.append(transition)
            observation = next_observation

        # Converte a lista de transições em np.array e adiciona à lista de episódios
        episodes_data.append(np.array(episode, dtype=object))
    
    env.close()
    # Retorna os episódios como um np.array (dtype=object, pois episódios podem ter comprimentos diferentes)
    return np.array(episodes_data, dtype=object)

episodes = collect_episodes(10)

In [ ]:
policy = PolicyNetwork(state_dim)

for iteration in range(iterations):
    reference_policy = policy
    
    for steps in range(steps):
        num_episodes = len(episodes)
        # Se num_samples for maior que o número total de episódios, usa todos
        num_samples = min(num_samples, num_episodes)
        
        # Seleciona índices de episódios aleatoriamente, sem repetição
        indices = np.random.choice(num_episodes, size=num_samples, replace=False)
        
        policy_old = policy
        
        